In [ ]:
import hashlib
from io import BytesIO
import random

import util
from test_framework.address import program_to_witness
from test_framework.key import ECKey, ECPubKey, SECP256K1_ORDER, generate_schnorr_nonce
from test_framework.messages import CTransaction, COutPoint, CTxIn, CTxOut, CScriptWitness, CTxInWitness, ser_string
from test_framework.musig import generate_musig_key, aggregate_schnorr_nonces, sign_musig, aggregate_musig_signatures
from test_framework.script import  CScript, OP_1, OP_CHECKSIG, TaprootSignatureHash, TapLeaf, TapTree, Node

# 2.2 TapTweak

* Part 1 - Tweaking the Public Key
* Part 2 - Commitment Schemes with Tweaks
* Part 3 - TapTweak Script Commitments

The linear property of bip-schnorr means that we can encode a commitment into a public key, and then reveal that commitment when signing with the private key. We do that by _tweaking_ the private key with the commitment, and using the associated _tweaked_ pubkey. When signing, we can reveal that the original private key was tweaked by the commitment.

## Part 1: Tweaking the Public Key

Instead of using our original public key as the witness program, we use a tweaked public key.

* `[01] [33B Tweaked Public Key]`

Tweaking a public key means to alter it with a value (the tweak) such that it remains spendable with knowledge of the original private key and tweak.

* `x` / `P`: Original private key / public key pair
* `t` / `T`: Tweak value / tweak point
* Output script: `[01] [P + T]` = `[01] [xG + tG]`
* Spendable by the tweaked private key: `x + t`

An observer cannot distinguish between a tweaked and untweaked public key.

#### Example 2.2.2: Signing with a tweaked keypair

In this example, we generate a key pair as before, and then tweak both the private key and public key. We then sign with the tweaked private key and verify that the signature is valid.

A _tweak_ is positive scalar value `t` where `0 < t < SECP256K1_ORDER`. There is an associated tweak point `T` such that `T = t*G`.

The private key is tweaked by the tweak scalar: `x' = x + t` and the public key is tweaked by the tweak point: `P' = P + T`.

The tweaked private key `x'` can be used to produce a valid signature for the tweaked pubkey `P'`.

In [ ]:
# Generate a key pair
privatekey = ECKey().generate()
publickey = privatekey.get_pubkey()

print("Private key: {}\nPublic key: {}\n".format(privatekey.secret, publickey.get_bytes().hex()))

# Generate a random tweak 0 < t < SECP256K1_ORDER and its associated point
tweak = random.randrange(1, SECP256K1_ORDER)
tweak_private = ECKey().set(tweak, True)
tweak_point = tweak_private.get_pubkey()
print("Tweak scalar: {}\nTweak point: {}\n".format(tweak_private.secret, tweak_point.get_bytes().hex()))

# Derive the tweaked private key and public key
privatekey_tweaked = privatekey + tweak_private
publickey_tweaked = publickey + tweak_point
print("Tweaked private key: {}\nTweaked pubkey: {}\n".format(privatekey_tweaked.secret, publickey_tweaked.get_bytes().hex()))

# Sign the message with tweaked key pair and verify the signature
msg = hashlib.sha256(b'msg').digest()
sig = privatekey_tweaked.sign_schnorr(msg)
assert publickey_tweaked.verify_schnorr(sig, msg)
print("Success!")

#### _Programming Exercise 2.2.2:_  Signing with a tweaked 2-of-2 MuSig key pair

In this exercise, we tweak an MuSig aggregate pubkey, and then sign for it using the individual participant keys. The MuSig pubkey aggregation step is done for you.

_Question: Which participant(s) need to tweak their private keys?_

In [ ]:
# Generate key pairs
privkey0 = ECKey().generate()
privkey1 = ECKey().generate()
pk0 = privkey0.get_pubkey()
pk1 = privkey1.get_pubkey()

# Create an aggregate MuSig pubkey
c_map, pk_musig = generate_musig_key([pk0, pk1])

# Apply challenge factors to keys
privkey0_c = privkey0.mul(c_map[pk0])
privkey1_c = privkey1.mul(c_map[pk1])
pk0_c = pk0.mul(c_map[pk0])
pk1_c = pk1.mul(c_map[pk1])
    
# Tweak musig public key
tweak = random.randrange(1, SECP256K1_ORDER)
pk_musig_tweaked =  # TODO: implement

# Nonce generation & aggregation
# Remember to negate the individual nonce values if required
R_agg = . # TODO: implement

# Signing and signature aggregation
msg = hashlib.sha256(b'msg').digest()

# Sign individually and then aggregate signatures
sig_agg =  # TODO: implement

assert pk_musig_tweaked.verify_schnorr(sig_agg, msg)
print("Success!")

## Part 2: Commitment schemes with tweaks

Taproot uses the tweak as a commitment for spending script paths. However, simply applying the committed value as a public key tweak is not sufficient, as this does not represent a secure cryptographic commitment.

![test](images/taptweak0.jpg)

Instead, the committed value must first be hashed with the untweaked public key point. **This prevents modification of both untweaked secret and tweak for a given tweaked pubkey point Q.**

#### Example 2.2.3: modifying the tweak for a tweaked public key Q

In this example we demonstrate an insecure commitment scheme. Simply tweaking the private key with a value `c` allows the pubkey equation `Q = x'G + c'G` to be solved for any `c'` by modifying `x'`.

In [ ]:
# Generate a key pair
x = ECKey().generate()
pk = x.get_pubkey()
print("Private key: {}\nPublic key: {}\n".format(x.secret, pk.get_bytes().hex()))

# Tweak the public key
t = random.randrange(1, SECP256K1_ORDER)
print("Tweak: {}".format(t))
Q = pk.tweak_add(t)

# Create a fake tweak
t2 = random.randrange(1, SECP256K1_ORDER)
print("Tweak 2: {}\n".format(t2))

# Solve for x` = x - (t' - t)
x_int = x.as_int()
x2_int = (x_int - t2 + t) % SECP256K1_ORDER

x2_key = x = ECKey().set(x2_int, True)
pk2 = x2_key.get_pubkey()
Q2 = pk2.tweak_add(t2)

print("Tweaked pubkey for x tweaked by t: {}".format(Q.get_bytes().hex()))
print("Tweaked pubkey for x2 tweaked by t2: {}".format(Q2.get_bytes().hex()))

#### Example 2.2.4 - Tweaking the pubkey with `H(P|msg)`

In this example, we demonstrate a _secure_ commitment scheme. The private key is tweaked with the scalar `H(P|c)`. Since `P` appears both inside and outside the hash, it isn't possible to solve for a different `c` by modifying `x'`.

In [ ]:
# Key pair generation
privkey = ECKey().generate()
publickey = privkey.get_pubkey()
print("Private key: {}\nPublic key: {}\n".format(x.secret, pk.get_bytes().hex()))

# Compute the tweak from H(P|msg)
commitment = b'commitment'
ss = hashlib.sha256(publickey.get_bytes()).digest()
ss += hashlib.sha256(commitment).digest()
t = hashlib.sha256(ss).digest()

# Determine tweak point
tweak_private = ECKey().set(t, True)
tweak_point = tweak_private.get_pubkey()
print("Tweak scalar: {}\nTweak point: {}\n".format(tweak_private.secret, tweak_point.get_bytes().hex()))

privkey_tweaked = privkey + tweak_private
publickey_tweaked = publickey + tweak_point

# Sign message and verify signature
msg = hashlib.sha256(b'msg').digest()
sig = privkey_tweaked.sign_schnorr(msg)

assert publickey_tweaked.verify_schnorr(sig, msg)
print("Success!")

### Taproot: Tagged Hashes.

The Taproot Proposal describes tagged hashes for both the taptree and taptweak. 

The TapTweak uses this double nested hashing function because it already used in TapBranches and TapLeafs, where it provides context uniqueness across the Bitcoin protocol. The 64B length of the two sha256 digest concatenation  lends itself to optimization in implementations. 

**Tagged Hash:**
* `tagged_hash` = `sha256(sha256("Tag") + sha256("Tag") + data)`


**TapTree Node Hashes:**
* `TapTweak` 
    * `= sha256(sha256("TapTweak") + sha256("TapTweak") + node_hash)`
* `TapBranch` 
    * `= sha256(sha256("TapBranch") + sha256("TapBranch") + node_hash_left|node_hash_right)`
* `TapLeaf` 
    * `= sha256(sha256("TapLeaf") + sha256("TapLeaf") + node_hash_left|node_hash_right)`
    
_Ordering of left and right node hash data is determined lexicographically, see next section._

## Part 3 - TapTweak Script Commitments

The TapTweak commits a Taptree to the segwit version 1 public key. It does so with a commitment structure resembling familiar merkle tree construction.

_Please note that the taptree uses tagged hashes which prevent node height ambiguity currently found in the transaction merkle tree, which allow an attacker to create a node which can be reinterpreted as either a leaf or internal node._


**The TapTree is different than the header merkle tree in the following ways:**

* Tapleafs can be located at different heights.
* Ordering of TapLeafs is determined lexicograpically.
* Location of nodes are tagged (No ambiguity of node type).
 
![test](images/taptweak1.jpg)


### Programming Exercise 3.1 - Constructing a TapTweak from TapScripts.

In the cell below, we will commit three pay-to-pubkey scripts to a taptweak and then derive the segwit address which can be spent by fulfilling these scriptpaths and the internal. We will use the same merkle tree structure as in the previous illustration.

* 1. Compute TapLeafs A, B and C.
* 2. Compute Internal node TapBranch AB.
* 3. Compute TapTweak
* 4. Derive the segwit output address.


In [ ]:
TAPSCRIPT_VER = bytes([0xc0]) # See TapScript chapter for more details.
internal_pubkey = ECPubKey()
internal_pubkey.set(bytes.fromhex('03af455f4989d122e9185f8c351dbaecd13adca3eef8a9d38ef8ffed6867e342e3'))

# Derive pay-to-pubkey scripts.
secA = ECKey()
secB = ECKey()
secC = ECKey()
secA.generate()
secB.generate()
secC.generate()
pkA = secA.get_pubkey()
pkB = secA.get_pubkey()
pkC = secA.get_pubkey()
scriptA = CScript([pkA.get_bytes(), OP_CHECKSIG])
scriptB = CScript([pkB.get_bytes(), OP_CHECKSIG])
scriptC = CScript([pkC.get_bytes(), OP_CHECKSIG])

# Method: Returns Tagged Hash.
def tagged_hash(tag, input_data):
    data = hashlib.sha256(tag.encode('utf-8')).digest()
    data += data
    data += input_data
    return hashlib.sha256(data).digest()

# Method: Returns TapBranch hash.
def tapbranch(taggedhash_left, taggedhash_right):
    if taggedhash_left > taggedhash_right:
        taggedhash_left, taggedhash_right = taggedhash_right, taggedhash_left
    return tagged_hash("TapBranch", taggedhash_left + taggedhash_right)  

# 1) Compute TapLeafs A, B and C.
# Note: ser_string(data) is a function which adds compactsize to input data.


# 2) Compute Internal node TapBranch AB.


# 3) Compute TapTweak.


# 4) Derive the segwit output address.



**Run the cell below to check if you have correctly computed the TapTweak.**

In [ ]:
# This code uses the TapTree and TapLeaf classes to construct the same tweak as above

tapleafA = TapLeaf()
tapleafB = TapLeaf()
tapleafC = TapLeaf()
taptree = TapTree()

tapleafA.from_script(scriptA)
tapleafB.from_script(scriptB)
tapleafC.from_script(scriptC)

tapbranchAB = Node()
tapbranchAB.left = tapleafA
tapbranchAB.right = tapleafB

tapbranchABC = Node()
tapbranchABC.left = tapbranchAB
tapbranchABC.right = tapleafC

taptree.root = tapbranchABC
taptree.key = internal_pubkey

script_v1, tweak, c_map = taptree.construct()
print("TapTweak:", tweak.hex())

## Example: Spending a taproot output along the key path

In this exercise, we'll create a segwit version 1 output that sends to a tweaked public key. We'll them spend that output along the key path using the tweaked private key.

Such as spend does not reveal the committed tweak to the observer and is indistinguishable any other key path spend.

**Start TestNodes**

In [ ]:
# Start TestWrapper
test = util.TestWrapper()
test.setup()

**Generate Wallet Balance**

In [ ]:
# Generate Coins for Bitcoin Node wallet
test.nodes[0].generate(101)
balance = test.nodes[0].getbalance()

print("Balance: {}".format(balance))

#### _Programming Exercise 2.2.5:_ Construct taproot output with tweaked public key

In [ ]:
# Example key pair
privatekey = ECKey().set(102118636618570133408735518698955378316807974995033705330357303547139065928052, True)
internal_pubkey = privatekey.get_pubkey()

# Example tweak
taptweak = bytes.fromhex('2a2fb476ec9962f262ff358800db0e7364287340db73e5e48db36d1c9f374e30')

# Tweak the public key
taproot_pubkey =  # TODO: implement
taproot_pubkey_b =  # TODO: implement

# Derive the bech32 address
segwit_address =  # TODO: implement

assert segwit_address == "bcrt1pq9lku0vuddzvcte8yvt3xct0dk6cjqeq2yzqp3vwpvh2e8afqpvqqyftl09"
print("Success! Segwit Address:".format(segwit_address))

#### Send funds from the wallet to the taproot output

In [ ]:
# Send funds to taproot output
txid = test.nodes[0].sendtoaddress(segwit_address, balance / 100000)
print("Funding tx:{}\n".format(txid))

# Deserialize wallet transaction
tx = CTransaction()
tx_hex = test.nodes[0].getrawtransaction(txid)
tx.deserialize(BytesIO(bytes.fromhex(tx_hex)))
tx.rehash()

# The wallet randomizes the change output index for privacy
# Loop through the outputs and return the first where the scriptPubKey matches the segwit v1 output
output_index, output = next(out for out in enumerate(tx.vout) if out[1].scriptPubKey == CScript([OP_1, taproot_pubkey_v1]))

print("Segwit v1 output is {}".format(output))
print("Segwit v1 output value is {}".format(output.nValue))
print("Segwit v1 output index is {}".format(output_index))

In [ ]:
# Construct transaction and fill version, locktime and inputs
tx2 = CTransaction()
tx2.nVersion = 1
tx2.nLockTime = 0
outpoint = COutPoint(tx.sha256, index)
tx2_in = CTxIn(outpoint = outpoint)
tx2.vin = [tx2_in]
print("CTransaction: {}\n".format(tx2))

# Generate new Bitcoin Core wallet address to send funds to
dest_addr = test.nodes[0].getnewaddress(address_type="bech32")
scriptpubkey = bytes.fromhex(test.nodes[0].getaddressinfo(dest_addr)['scriptPubKey'])
print("Sending to address: {}".format(dest_addr))

# Determine minimum fee required for mempool acceptance
min_fee = int(test.nodes[0].getmempoolinfo()['mempoolminfee'] * 100000000)

#### _Programming Exercise 2.2.6:_ Spend taproot output with key path

In [ ]:
# Complete output which returns funds to Bitcoin Core wallet.
dest_output = CTxOut(nValue=output_value-min_fee, scriptPubKey=scriptpubkey)
tx2.vout = [dest_output]

# Sign transaction with tweaked private key
sig =  # TODO: implement

# Construct transaction witness
witness = CScriptWitness()
tx2.wit.vtxinwit.append(  # TODO: implement

# Serialize transaction for broadcast
tx2_str =  # TODO: implement

# Test mempool acceptance
assert test.nodes[0].testmempoolaccept([tx2_str])[0]['allowed']
print("Success!")

#### Shutdown TestWrapper

In [ ]:
# Shutdown
test.shutdown()